In [22]:
import folium
from folium.features import DivIcon
import shapely
from shapely.geometry import shape
import pickle
import pandas as pd
import pyproj
import shapely.wkt
from shapely.ops import transform

In [23]:
file_name = 'street_info.bin'
df = pd.read_pickle(file_name)
wgs84 = pyproj.CRS('EPSG:4326')
fl_sp_east_feet = pyproj.CRS('EPSG:2236')
project = pyproj.Transformer.from_crs(fl_sp_east_feet, wgs84, always_xy=True).transform  # note from_epgg, to_epsg order


In [24]:
# start our map over ERAU.
m = folium.Map(location=[29.1887876219045, -81.0494807582431], zoom_start=18, width="100%")

def highlight_function(feature):  # how to color the features when highlighted
    return{
        'weight': 5,
        'color': '#626',
        'dashArray': '',
        'fillOpacity': 0.7
    };

def style_function(feature):     # how to color the features we are adding to the map
    return{
        'weight': 1.5,
        #'color': 'blue',
        'dashArray': '5, 5',
        'fillOpacity': 0.7
    };

fgv = folium.FeatureGroup(name="parcels")   # create a folium map layer feature group, then add polygons, then add to map m
fgv2 = folium.FeatureGroup(name="parcel_labels")   # create a folium map layer feature group, then add polygons, then add to map m

for index, row in df.iterrows():    
    #tooltip_show = str(int(row.adrno))      # 
    geom = shapely.wkt.loads(row.geometry)  # use shapely wkt.loads to load WKT into a shapeply geometry (polygon)
    wgs_geom = transform(project, geom)     # reproject that polygon to wgs84
    
    # create geoJSON Features, add a popup to them, then add to the feature group
    c= folium.GeoJson(shapely.geometry.asShape(wgs_geom),  # convert the wgs84 shapely geom to JSON and add to group 
                       name='parcels',      # add all these geometries to feature group named parcels?
                       overlay=True,
                       style_function=style_function,  # how to color the feature
                       highlight_function=highlight_function, # how to color/symbolize on highlight/mouseover
                       tooltip=tooltip_show   # on hover over show this information,
                       #zoom_on_click=True     # does not work on my version of folium
    )
    # generate a folium Popup then add it to the GeoJson object c, format all data into a table
    popup_string = '<table>'
    for i, v in row.iteritems():
        popup_string += "<tr><td>%10s</td><td>&nbsp;" % i.title() + " " + str(v) + "</td></tr>\n"
    popup_string += "</table>"
    folium.Popup(popup_string).add_to(c)    # add this popup to the c object
    
    # now that we've created a GeoJson folium feature, with popups, add it to the feature group
    fgv.add_child(c)
    
    # We also want to add a label for each parcel on the map. Create a separate folium Markers for each label
    # since we already have the the shapely library loaded, can quickly get access to the geometries centroid
    # grab the lat, lon of the centroid of each parcel and add to map as a Marker
    # https://stackoverflow.com/questions/46400769/numbers-in-map-marker-in-folium
    # have to put at centroid, format text as html div
    divlabel = '<div style="font-size: 10pt; color : black">' + tooltip_show + '</div>'
    marker = folium.Marker(
        [wgs_geom.centroid.y, wgs_geom.centroid.x], 
        icon=DivIcon(html=divlabel)
    ).add_to(fgv2)   # added to second feature group, so that we can control separately

# add created map layers to map
fgv.add_to(m)    # now add the whole feature group to the map, which we call parcels
fgv2.add_to(m)   # add parcel_labels

m.add_child(folium.LayerControl(position='topright', collapsed=False))

m.fit_bounds(fgv.get_bounds())  # set the map bounds to the feature group bounds  wgs84 



In [26]:
m